In [1]:
import re
import time
import folium
import numpy as np
import pandas as pd
from geopy.geocoders import ArcGIS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Property_Location.csv')

In [3]:
df["Location"] = df["Location"].str.replace(' ','+')
df["Region"] = df["Region"].str.replace(' ','+')

In [7]:
print("Shape of Data :",df.shape)
df.head()

Shape of Data : (2580, 13)


,Property_Name,Location,Price,Rate_SqFt,Area_Tpye,Bedroom,Bathroom,Floor_No,Property_Age,Availability,Area_SqFt,Region,Price_Lakh
0,Omkar Alta Monte,W+E+Highway+Malad+East+Mumbai,5 Crore,17241,Super Built Up Area,3,4,14,0 to 1 Year,Ready To Move,2900.0,Highway+Malad,500.0
1,T Bhimjyani Neelkanth Woods,Manpada+Thane+Mumbai,2.4 Crore,12631,Super Built Up Area,3,3,8,1 to 5 Year,Ready To Move,1900.0,Manpada+Thane,240.0
2,Legend 1 Pramila Nagar,Dahisar+West+Mumbai,95 Lac,15966,Super Built Up Area,1,2,3,10+ Year,Ready To Move,595.0,Dahisar+Mumbai,95.0
3,Unnamed Property,Vidyavihar+West+Vidyavihar+West+Central+Mumbai...,3.75 Crore,25862,Built Up Area,3,3,1,5 to 10 Year,Ready To Move,1450.0,Central+Mumbai,375.0
4,Unnamed Property,176+Cst+Road+Kalina+Mumbai+400098+Santacruz+Ea...,3.5 Crore,39954,Carpet Area,2,2,5,5 to 10 Year,Ready To Move,876.0,Santacruz+Mumbai,350.0


In [11]:
location = []
def Location_Locate():
    for i in range(0,len(df["Location"])):
        try:
            locator = Nominatim(user_agent="streamlitdemo")
            geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
            location.append(geocode(df['Location'][i]))
        except:
            locator = Nominatim(user_agent="streamlitdemo")
            geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
            location.append(geocode(df['Region'][i]))
        else:
            pass
Location_Locate()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Panchpakhadi+Thane+West+Thane+West+Thane+Mumbai',), **{}).
Traceback (most recent call last):
  File "C:\Users\shanu\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\shanu\anaconda3\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\shanu\anaconda3\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\shanu\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\shanu\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\Users\shanu\anaconda3\li

In [12]:
Latitude = []
Longitude = []
for i in range(0,len(location)):
    if(location[i]):
        Latitude.append(location[i].latitude)
        Longitude.append(location[i].longitude)
    else:
        Latitude.append(np.nan)
        Longitude.append(np.nan)
print("Total Latitude : ", len(Latitude))

Total Latitude :  2580


#### Adding Latitude and Longitude to the dataframe

In [13]:
df['Latitude'] = Latitude
df['Longitude'] = Longitude

In [14]:
df["Location"] = df["Location"].str.replace('+', ' ')
df["Region"] = df["Region"].str.replace('+', ' ')

In [15]:
df.head()

,Property_Name,Location,Price,Rate_SqFt,Area_Tpye,Bedroom,Bathroom,Floor_No,Property_Age,Availability,Area_SqFt,Region,Price_Lakh,Latitude,Longitude
0,Omkar Alta Monte,W E Highway Malad East Mumbai,5 Crore,17241,Super Built Up Area,3,4,14,0 to 1 Year,Ready To Move,2900.0,Highway Malad,500.0,NaN,NaN
1,T Bhimjyani Neelkanth Woods,Manpada Thane Mumbai,2.4 Crore,12631,Super Built Up Area,3,3,8,1 to 5 Year,Ready To Move,1900.0,Manpada Thane,240.0,19.235319,72.975950
2,Legend 1 Pramila Nagar,Dahisar West Mumbai,95 Lac,15966,Super Built Up Area,1,2,3,10+ Year,Ready To Move,595.0,Dahisar Mumbai,95.0,19.064823,72.837362
3,Unnamed Property,Vidyavihar West Vidyavihar West Central Mumbai...,3.75 Crore,25862,Built Up Area,3,3,1,5 to 10 Year,Ready To Move,1450.0,Central Mumbai,375.0,NaN,NaN
4,Unnamed Property,176 Cst Road Kalina Mumbai 400098 Santacruz Ea...,3.5 Crore,39954,Carpet Area,2,2,5,5 to 10 Year,Ready To Move,876.0,Santacruz Mumbai,350.0,NaN,NaN


In [16]:
df.isna().sum()

Property_Name       0
Location            0
Price               0
Rate_SqFt           0
Area_Tpye           0
Bedroom             0
Bathroom            0
Floor_No            0
Property_Age        0
Availability        0
Area_SqFt           0
Region              0
Price_Lakh          0
Latitude         1567
Longitude        1567
dtype: int64

In [17]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [18]:
df.shape

(1013, 15)

In [19]:
df.to_csv('Map_Location.csv', index=False)

In [20]:
df2 = pd.read_csv('Map_Location.csv')
df2

,Property_Name,Location,Price,Rate_SqFt,Area_Tpye,Bedroom,Bathroom,Floor_No,Property_Age,Availability,Area_SqFt,Region,Price_Lakh,Latitude,Longitude
0,T Bhimjyani Neelkanth Woods,Manpada Thane Mumbai,2.4 Crore,12631,Super Built Up Area,3,3,8,1 to 5 Year,Ready To Move,1900.0,Manpada Thane,240.0,19.235319,72.975950
1,Legend 1 Pramila Nagar,Dahisar West Mumbai,95 Lac,15966,Super Built Up Area,1,2,3,10+ Year,Ready To Move,595.0,Dahisar Mumbai,95.0,19.064823,72.837362
2,Varun Garden,Manpada Thane Mumbai,2.2 Crore,14666,Super Built Up Area,3,3,7,5 to 10 Year,Ready To Move,1500.0,Manpada Thane,220.0,19.235319,72.975950
3,Green Court,702 Andheri West Mumbai,2.75 Crore,34810,Built Up Area,2,2,7,5 to 10 Year,Ready To Move,1200.0,Andheri Mumbai,275.0,19.117249,72.833968
4,Balaji Galaxy,502 Kalamboli Navi Mumbai Mumbai,70 Lac,11111,Carpet Area,2,2,5,10+ Year,Ready To Move,630.0,Kalamboli Navi-Mumbai,70.0,19.023506,73.110609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,Unnamed Property,Koparkhairane Navi Mumbai Mumbai,85 Lac,20481,Carpet Area,1,1,4,10+ Year,Ready To Move,415.0,Koparkhairane Navi-Mumbai,85.0,19.100837,72.998541
1009,Balaji Mayuresh Delta,Ulwe Navi Mumbai Mumbai,1.78 Crore,9569,Built Up Area,3,3,8,1 to 5 Year,Ready To Move,1860.0,Ulwe Navi-Mumbai,178.0,18.976861,73.020131
1010,Balaji Mayuresh Delta,Ulwe Navi Mumbai Mumbai,1.37 Crore,10579,Built Up Area,2,2,6,1 to 5 Year,Ready To Move,1295.0,Ulwe Navi-Mumbai,137.0,18.976861,73.020131
1011,Balaji Mayuresh Delta,Ulwe Navi Mumbai Mumbai,1.71 Crore,9243,Built Up Area,3,3,6,1 to 5 Year,Ready To Move,1850.0,Ulwe Navi-Mumbai,171.0,18.976861,73.020131


In [21]:
lat = list(df2['Latitude'])
lon = list(df2['Longitude'])
name = list(df2['Property_Name'])
add = list(df2['Location'])
price = list(df2['Price'])
age = list(df2['Property_Age'])
avail = list(df2['Availability'])
rate = list(df2['Rate_SqFt'])

gmap = folium.Map(location=[19.25029770723734, 73.13414632644128], zoom_start=10)
fg = folium.FeatureGroup(name="My Map").add_to(gmap)

In [22]:
for lat,lon,name,add,price,age,avail,rate in zip(lat,lon,name,add,price,age,avail,rate):
    html = f"""<p style="color:green" > Property Name : {name}<p/>
               <p style="color:green" > Address : {add}<p/>
               <p style="color:red" > Price : {price}<p/>
               <p style="color:red" > Rate SqFt : {rate}<p/>
               <p style="color:blue" > Property Age : {age}<p/>
               <p style="color:blue" > Availability : {avail}<p/>
            """
    iframe = folium.IFrame(html, width=250, height=270)
    popup = folium.Popup(iframe, max_width=300)
    marker = folium.Marker([lat,lon], popup=(popup)).add_to(gmap)
    gmap.add_child(marker)

gmap.add_child(fg)
gmap.save("mumbai_property.html")

In [23]:
gmap.add_child(fg)

# The End !!